<a href="https://colab.research.google.com/github/vifirsanova/compling/blob/main/RBMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Машинный перевод на основе правил

## Архитектура системы машинного перевода на основе правил

## DIY-модель

Базовая модель

Добавляем правила в базовую модель

Создаем языковые правила

## Apertium

Что такое Apertium?

### APY Apertium

Что такое APY?

Переводим с помощью APY

### Компоненты Apertium

Анализатор

In [ ]:
# запускаем

Как работает анализатор?

Генератор

In [ ]:
# запускаем

Как работает генератор?

Теггер

In [ ]:
# запускаем

Как работает теггер?

Перевод

In [ ]:
# запускаем

Как работает перевод?

# Домашнее задание

Представьте, что вы работаете аналитиком в проекте по разработке системы машинного перевода. Ваша команда создает систему машинного перевода на основе правил. Прежде чем ваши коллеги приступят к разработке системы, вам, как аналитику, нужно описать архитектуру будущей системы.

Опишите структуру своей системы машинного перевода на основе правила в формате псевдокода. В процессе работы вам нужно будет ответить на следующие вопросы:

+ Какую языковую пару обрабатывает ваша система?
+ Какие виды анализа и предварительной обработки данных вы применяете?
+ Какие языковые правила вам нужно описать?
+ Как ваша система будет генерировать результат, т.е. перевод?

In [ ]:
# образец псевдокода